In [5]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [7]:
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [8]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=groq_api_key, 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36827")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "Lead Software Engineer, ITC",
  "experience": "8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications",
  "skills": [
    "PL/SQL",
    "Cloud environments (AWS, Azure or GCP)",
    "Scrum and other Agile processes",
    "Jira, Git/SVN, Jenkins, CI/CD pipelines",
    "Integrated Business Planning solutions like O9",
    "Planning business domain, data domains, planning algorithms and solvers"
  ],
  "description": "Lead a team of engineers to develop solutions for Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation."
}
```


In [19]:
type(res.content)

str

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications',
 'skills': ['PL/SQL',
  'Cloud environments (AWS, Azure or GCP)',
  'Scrum and other Agile processes',
  'Jira, Git/SVN, Jenkins, CI/CD pipelines',
  'Integrated Business Planning solutions like O9',
  'Planning business domain, data domains, planning algorithms and solvers'],
 'description': 'Lead a team of engineers to develop solutions for Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation.'}

In [20]:
type(json_res)

dict

In [21]:
import pandas as pd

data = pd.read_csv("my_portfolio.csv")
data

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [22]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in data.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [23]:
results = collection.query(
    query_texts=["Query is about python"],
    n_results=3
)
results

{'ids': [['e62e6b5d-1522-4c26-ac99-1a2d523da881',
   '914168d2-93c4-4926-8301-47feccd688d6',
   '162b1069-d96e-4294-9a36-ba3756e6cfa9']],
 'distances': [[0.9077300459725457, 1.4446809360551678, 1.4681162837919177]],
 'metadatas': [[{'links': 'https://example.com/python-portfolio'},
   {'links': 'https://example.com/ml-python-portfolio'},
   {'links': 'https://example.com/magento-portfolio'}]],
 'embeddings': None,
 'documents': [['Python, Django, MySQL',
   'Machine Learning, Python, TensorFlow',
   'Magento, PHP, MySQL']],
 'uris': None,
 'data': None}

In [27]:
job = json_res
job['skills']

['PL/SQL',
 'Cloud environments (AWS, Azure or GCP)',
 'Scrum and other Agile processes',
 'Jira, Git/SVN, Jenkins, CI/CD pipelines',
 'Integrated Business Planning solutions like O9',
 'Planning business domain, data domains, planning algorithms and solvers']

In [28]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}]]

In [30]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Zubair, a business development executive at Mirafra. Mirafra is an AI & Software company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Mirafra 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Mirafra's portfolio: {link_list}
        Remember you are Zubair, BDE at Mirafra. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Demand & Supply Planning Capability

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer, ITC, and I was impressed by the scope of the role. As a Business Development Executive at Mirafra, I'd like to introduce our company as a potential partner in developing innovative solutions for your Demand & Supply Planning capability.

At Mirafra, we specialize in facilitating seamless integration of business processes through automated tools, leveraging AI and software expertise. Our team of experts has a proven track record of empowering enterprises with tailored solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency.

Our capabilities align perfectly with your requirements, particularly in the areas of:

* Technical architecture, design, development, and deployment of demand or inventory planning products/applications
* Cloud environments (AWS, Azure, or GCP)
* Scrum and other 